In [39]:
import pandas as pd 

train = pd.read_csv("score_train.csv")
test = pd.read_csv("score_test.csv")

train.shape, test.shape


((4198, 21), (1499, 20))

In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4198 entries, 0 to 4197
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       4198 non-null   float64
 1   Num_of_Delayed_Payment    4198 non-null   float64
 2   Num_Credit_Inquiries      4198 non-null   float64
 3   Credit_Utilization_Ratio  4198 non-null   float64
 4   Credit_History_Age        4198 non-null   float64
 5   Payment_of_Min_Amount     4198 non-null   object 
 6   Amount_invested_monthly   4198 non-null   float64
 7   Monthly_Balance           4198 non-null   float64
 8   Credit_Mix                4198 non-null   object 
 9   Payment_Behaviour         4198 non-null   object 
 10  Age                       4198 non-null   float64
 11  Annual_Income             4198 non-null   float64
 12  Num_Bank_Accounts         4198 non-null   float64
 13  Num_Credit_Card           4198 non-null   float64
 14  Interest

In [41]:
train.describe()

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Amount_invested_monthly,Monthly_Balance,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
count,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000,4198.000000
mean,21.363983,13.353978,5.746308,32.231895,222.225345,191.510951,404.255307,33.313721,50484.607537,5.424726,5.530729,14.578371,3.563125,4195.260793,10.357006,1409.436679,109.368995
std,14.760722,6.189640,3.853921,5.139842,99.764260,193.237278,211.622208,10.736982,38079.236375,2.555354,2.062956,8.568630,2.465621,3159.974294,6.532086,1146.835939,141.802993
min,0.000000,0.000000,0.000000,20.985606,1.000000,0.000000,8.063863,14.000000,7019.435000,0.000000,1.000000,1.000000,0.000000,357.255833,0.510000,0.540000,0.000000
25%,10.000000,9.000000,3.000000,27.990565,148.000000,71.267456,272.249521,24.000000,19558.610000,4.000000,4.000000,8.000000,2.000000,1654.409375,5.400000,546.840000,29.294379
50%,19.000000,14.000000,5.000000,32.117580,221.000000,127.766968,340.080970,33.000000,36820.590000,6.000000,5.000000,13.000000,3.000000,3088.630000,9.275000,1170.415000,68.084075
75%,28.000000,18.000000,8.000000,36.488230,302.000000,234.452093,474.931229,42.000000,71842.800000,7.000000,7.000000,20.000000,5.000000,5987.795833,14.910000,1916.575000,147.166548
max,67.000000,28.000000,17.000000,48.023249,404.000000,1733.700474,1508.926823,56.000000,179193.760000,10.000000,11.000000,34.000000,9.000000,15101.940000,29.980000,4990.910000,1779.103254


In [42]:
train.isnull().sum()

Delay_from_due_date         0
Num_of_Delayed_Payment      0
Num_Credit_Inquiries        0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Amount_invested_monthly     0
Monthly_Balance             0
Credit_Mix                  0
Payment_Behaviour           0
Age                         0
Annual_Income               0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Monthly_Inhand_Salary       0
Changed_Credit_Limit        0
Outstanding_Debt            0
Total_EMI_per_month         0
Credit_Score                0
dtype: int64

In [43]:
target = train.pop("Credit_Score")


In [44]:
cols = train.select_dtypes(include="object").columns
cols

Index(['Payment_of_Min_Amount', 'Credit_Mix', 'Payment_Behaviour'], dtype='object')

In [45]:
cols_ex = train.select_dtypes(exclude="object").columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train[cols_ex] = scaler.fit_transform(train[cols_ex])
test[cols_ex] = scaler.transform(test[cols_ex])

In [46]:
train[cols].nunique()

Payment_of_Min_Amount    3
Credit_Mix               3
Payment_Behaviour        6
dtype: int64

In [47]:
data = pd.concat([train, test])

data_oh = pd.get_dummies(data)

train = data_oh.iloc[:len(train)].copy()
test = data_oh.iloc[len(train):].copy()

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train, 
    target, 
    test_size= 0.2, 
    random_state=2025
)



In [49]:
from sklearn.ensemble import RandomForestClassifier 
import lightgbm as lgb 


rf = RandomForestClassifier(random_state=2025)
rf.fit(X_train, y_train) 
pred_rf = rf.predict(X_val)

lgbmc = lgb.LGBMClassifier(random_state=2025, verbose=-1)
lgbmc.fit(X_train, y_train)
pred_lgb = lgbmc.predict(X_val)

In [50]:
from sklearn.metrics import f1_score

f1_rf = f1_score(y_val, pred_rf, average="macro")
f1_lgb = f1_score(y_val, pred_lgb, average="macro")

f1_rf, f1_lgb


(0.7100428447955952, 0.7021258864294014)

In [51]:
pred=  rf.predict(test)
pred

array(['Poor', 'Good', 'Standard', ..., 'Standard', 'Good', 'Standard'],
      shape=(1499,), dtype=object)

In [52]:
train.shape, test.shape, data.shape

((4198, 29), (1499, 29), (5697, 20))